In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import math
import seaborn as sns
from cvx.covariance.combination import from_ewmas


In [ ]:
#Ensemble des librairies utilisées
from xbbg import blp
import datetime
import numpy as np
import pandas as pd
import time
import os
import tensorflow as tf
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy import stats


from pylatex import Document, Section, Subsection, Math, Table, Tabular, Figure, Command, MiniPage
from pylatex.utils import NoEscape
import subprocess
from pylatex.utils import bold
from pylatex.package import Package
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

from plotly.subplots import make_subplots
from tensorflow.keras.initializers import GlorotUniform
from keras.layers import Dropout
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from statsmodels.tsa.stattools import adfuller
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [53]:
df=pd.read_csv(r'C:\Users\33640\OneDrive\Documents\GitHub\Covariance-Correlation-Volatility-Forecasting\SP500_daily_returns.csv')

In [54]:
nan_counts=df.isna().sum()
columns_to_keep = nan_counts[nan_counts <= 3].index
returns = df[columns_to_keep]
returns=returns.dropna()
returns.shape

(6224, 355)

In [55]:
dates=returns['Date']
returns=returns.drop('Date', axis=1)

In [56]:
def histo_cov(df, date, window):
    cov=np.zeros((len(df.columns),len(df.columns)))
    for i in range(date-window, date):
        cov+=np.outer(np.array(df.iloc[i,:]),np.array(df.iloc[i,:]))/(window)
    covs=pd.DataFrame(cov*250, columns=df.columns, index=df.columns)
    return(covs)

In [57]:
def ewma_cov(df, date, window, beta):
    cov=np.zeros((len(df.columns),len(df.columns)))
    for i in range(date-window, date):
        cov=(beta)*cov+(1-beta)*np.outer(np.array(df.iloc[i,:]),np.array(df.iloc[i,:]))
    covs=pd.DataFrame(cov*250, columns=df.columns, index=df.columns)
    return(covs)

In [58]:
halflife_pairs=[(10, 21), (21, 63), (63, 125)]
def ewma_cov(df, date, halflife_pairs, window):    
    combinator = from_ewmas(df.iloc[max(0,date-1000):date,:],
                                halflife_pairs,
                                min_periods_vola=window,  # min periods for volatility estimation
                                min_periods_cov=window)  # min periods for correlation estimation

    # Solve combination problem and loop through combination results to get predictors
    covariance_predictors = {}
    for predictor in combinator.solve(window=10):  # lookback window for optimization
        # From predictor we can access predictor.time, predictor.mean (=0 here),
        # predictor.covariance, and predictor.weights
        covariance_predictors[predictor.time] = predictor.covariance
    return(covariance_predictors[date]*250)
    

In [65]:
cov=ewma_cov(returns, 3000, halflife_pairs, 500)

In [66]:
cov

,A,AAPL,ABT,ACGL,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WM,WMB,WMT,WRB,WST,WY,XEL,XOM,YUM,ZBRA
A,0.307492,0.081876,0.061632,0.084893,0.136760,0.125311,0.124871,0.103233,0.202491,0.081147,...,0.106054,0.149825,0.048280,0.099817,0.131699,0.157546,0.065906,0.102532,0.094296,0.146915
AAPL,0.081876,0.081877,0.020323,0.032875,0.058121,0.051016,0.051185,0.042361,0.088958,0.031990,...,0.040080,0.067595,0.017952,0.036287,0.045181,0.056530,0.023848,0.042743,0.043067,0.059730
ABT,0.061632,0.020323,0.037423,0.024934,0.036210,0.031578,0.039333,0.034649,0.050776,0.029607,...,0.033782,0.041955,0.019628,0.028657,0.036958,0.043766,0.025365,0.033440,0.030317,0.039716
ACGL,0.084893,0.032875,0.024934,0.069624,0.053126,0.043499,0.052463,0.044552,0.075387,0.040425,...,0.048268,0.059805,0.024008,0.060166,0.052736,0.056108,0.031683,0.042798,0.037545,0.059023
ADBE,0.136760,0.058121,0.036210,0.053126,0.140532,0.080936,0.071426,0.066997,0.143927,0.050337,...,0.064961,0.084565,0.031220,0.066686,0.075519,0.092044,0.039625,0.062030,0.057760,0.089100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WY,0.157546,0.056530,0.043766,0.056108,0.092044,0.088986,0.089490,0.073103,0.134971,0.055620,...,0.067337,0.104776,0.036961,0.072080,0.087510,0.156827,0.045022,0.074702,0.064750,0.097644
XEL,0.065906,0.023848,0.025365,0.031683,0.039625,0.034999,0.044462,0.036461,0.054729,0.039986,...,0.038138,0.049792,0.020169,0.037237,0.037276,0.045022,0.037015,0.036697,0.030930,0.044260
XOM,0.102532,0.042743,0.033440,0.042798,0.062030,0.058953,0.070965,0.054311,0.089435,0.044139,...,0.060272,0.085821,0.028190,0.051829,0.061383,0.074702,0.036697,0.076495,0.044778,0.074227
YUM,0.094296,0.043067,0.030317,0.037545,0.057760,0.051824,0.054226,0.046694,0.080789,0.037829,...,0.045413,0.067020,0.025365,0.045548,0.054223,0.064750,0.030930,0.044778,0.077794,0.061580


In [7]:
class Cov_Models:
    def __init__(self, df, annualization):
        self.df = df
        self.annualization = annualization 
    def histo_cov(self, window):
        if window == 1:
            # Approxime la covariance en utilisant le produit des rendements pour chaque paire d'actifs
            cov_matrix = pd.DataFrame(index=self.df.columns, columns=self.df.columns)
            for col1 in self.df.columns:
                for col2 in self.df.columns:
                    cov_matrix.loc[col1, col2] = self.df.iloc[-1][col1] * self.df.iloc[-1][col2] * self.annualization
        else:
            # Calcul standard de la covariance sur la fenêtre définie
            cov_matrix = self.df.iloc[len(self.df.index) - window:len(self.df.index), :].cov() * self.annualization

        return cov_matrix
    
    def EWMA_series(self,beta, actif1,actif2):
        ewma_cov=0
        m1 = max(self.df.loc[:, actif1].first_valid_index(), self.df.loc[:, actif2].first_valid_index())
        while self.df.loc[m1:, actif1].isna().any() or self.df.loc[m1:, actif2].isna().any():
            m1+=1
        for t in range(m1,len(self.df.index)):
            r_t1 = self.df.loc[t,actif1]
            r_t2 = self.df.loc[t,actif2]
            ewma_cov=beta*ewma_cov+(1-beta)* r_t1*r_t2
        ewma_cov=ewma_cov*self.annualization
        return (ewma_cov)
    def EWMA_cov_opti(self, actif1,actif2, horizon):
        betas = np.linspace(0.90, 0.99, 21)
        covs = [[] for _ in range(len(betas))]
        score= [[] for _ in range(len(betas))]
        for j, beta in enumerate(betas):
            cov_realized = []
            for t in range(len(self.df.index)-100-horizon,len(self.df.index)-horizon, 10):
                realized=0
                for j in range(horizon):
                    realized+= self.df.loc[t+j:t + j, actif1]* self.df.loc[t+j:t + j, actif2] * self.annualization
                cov_realized.append(realized)
                covs[j].append(self.EWMA_series(beta, actif1,actif2))
            score[j]=np.sum((np.array(cov_realized)-np.array(covs[j]))**2)
        beta_opt=betas[score.index(min(score))]
        return(beta_opt)
    def EWMA_cov(self, beta_var, beta_cov, horizon):
        ewma_cov=[]
        pairs=[]
        for j in range(len(self.df.columns)):
            for i in range(j, len(self.df.columns)):
                pairs.append((self.df.columns[j], self.df.columns[i]))
        for j, (actif1, actif2) in enumerate (pairs):
            if beta_var=='Opti' and beta_cov=='Opti':
                ewma_cov.append(self.EWMA_series(self.EWMA_cov_opti( actif1,actif2, horizon), actif1,actif2))
            else:
                if actif1==actif2:
                    ewma_cov.append(self.EWMA_series(beta_var, actif1,actif2))
                else:
                    ewma_cov.append(self.EWMA_series(beta_cov, actif1,actif2))
        covs=np.zeros((len(self.df.columns),len(self.df.columns)))
        k=0
        for j in range(len(self.df.columns)):
            for i in range(j, len(self.df.columns)):
                covs[j,i]=covs[i,j]=ewma_cov[k]
                k+=1
        covariance=pd.DataFrame(covs, columns=self.df.columns, index=self.df.columns)
        return covariance
    
    def HAR_cov(self, windows_var, windows_cov, horizon):
        Har_covs = []
        pairs = []
        for j in range(len(self.df.columns)):
            for i in range(j, len(self.df.columns)):
                pairs.append((self.df.columns[j], self.df.columns[i]))
        for actif1, actif2 in pairs:
            m1 = max(self.df.loc[:, actif1].first_valid_index(), self.df.loc[:, actif2].first_valid_index())
            while self.df.loc[m1:, actif1].isna().any() or self.df.loc[m1:, actif2].isna().any():
                m1 += 1
            if actif1 == actif2:
                windows = windows_var
            else:
                windows = windows_cov
            covs = [[] for _ in range(len(windows))]
            cov_realized = []
            for t in range(max(max(windows[-1], m1),len(self.df.index) - 250),  len(self.df.index) - horizon):
                for i, window in enumerate(windows):
                    if window > 1:
                        cov_value = self.df.loc[t - window:t - 1, [actif1, actif2]].cov().iloc[0, 1] * self.annualization
                    else:
                        # Calcul de covariance pour une fenêtre de taille 1
                        cov_value = self.df.loc[t - 1, actif1] * self.df.loc[t - 1, actif2] * self.annualization
                    covs[i].append(cov_value)

                # Calcul pour cov_realized
                if horizon > 1:
                    cov_realized_value = self.df.loc[t:t + horizon, [actif1, actif2]].cov().iloc[0, 1] * self.annualization
                else:
                    cov_realized_value = self.df.loc[t, actif1] * self.df.loc[t, actif2] * self.annualization
                cov_realized.append(cov_realized_value)

            cov_data = pd.DataFrame({f'cov_{i+1}': covs[i] for i in range(len(windows))})
            cov_data['cov_realized'] = cov_realized
            cov_data = cov_data.dropna()

            X = cov_data[[f'cov_{i+1}' for i in range(len(windows))]]
            X = add_constant(X)
            y = cov_data['cov_realized']
            model = OLS(y, X).fit()
            latest_covs = {'intercept': [1]}
            for i, window in enumerate(windows):
                if window > 1:
                    latest_cov_value = self.df.loc[len(self.df.index) - window:len(self.df.index) - 1, [actif1, actif2]].cov().iloc[0, 1] * self.annualization
                else:
                    latest_cov_value = self.df.loc[len(self.df.index) - 1, actif1] * self.df.loc[len(self.df.index) - 1, actif2] * self.annualization
                latest_covs[f'cov_{i+1}'] = [latest_cov_value]

            lastest_data = pd.DataFrame(latest_covs)
            cov_forecast = model.predict(lastest_data)
            Har_covs.append(cov_forecast.iloc[0])
            if progress_bar:  # Update the progress bar if provided
                progress_bar.update(1)
        covs = np.zeros((len(self.df.columns), len(self.df.columns)))
        k = 0
        for j in range(len(self.df.columns)):
            for i in range(j, len(self.df.columns)):
                covs[j, i] = covs[i, j] = Har_covs[k]
                k += 1
        covariance = pd.DataFrame(covs, columns=self.df.columns, index=self.df.columns)
        return covariance
        
    def timeseries_to_supervised(self, data, lag=1):
        supervised_data = pd.DataFrame(data)
        columns = [supervised_data.shift(i) for i in range(1, lag+1)]
        columns.append(supervised_data)
        supervised_data = pd.concat(columns, axis=1)
        supervised_data.fillna(0, inplace=True)
        supervised_data = supervised_data[2:].reset_index(drop=True)
        return supervised_data

    def fit_lstm(self, X, y, batch_size, nb_epoch, neurons):
        # Define the model with the preferred Input(shape) object
        model = Sequential()
        model.add(Input(shape=(X.shape[1], X.shape[2])))  # Using Input layer to define input shape
        model.add(LSTM(neurons))
        model.add(Dropout(0.2))
        model.add(Dense(1))

        # Compile the model
        model.compile(loss='mean_squared_error', optimizer='adam')

        # Fit the model
        for _ in range(nb_epoch):
            model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)

        return model

    def LSTM_cov(self,  batch_size, nb_epoch, neurons, horizon):
        cov = []
        pairs=[]
        Stationnary={}
        for actif in self.df.columns:
            rendements=self.df.loc[:,actif]
            m1=rendements.first_valid_index()
            while rendements.iloc[m1:].isna().any():
                m1+=1
            y=rendements.iloc[m1:]
            adf_result = adfuller(y)
            if adf_result[1]>=0.05:
                Stationnary[actif]=False
            else:
                Stationnary[actif]=True
        k=0
        for j in range(len(self.df.columns)):
            for i in range(j, len(self.df.columns)):
                k+=1
                pairs.append((self.df.columns[j], self.df.columns[i]))
        k=0
        for actif1, actif2 in pairs:
            if Stationnary[actif1]==False or Stationnary[actif2]==False:
                cov.append(self.df.loc[len(self.df.index)-horizon*self.annualization:len(self.df.index), [actif1,actif2]].cov().iloc[0,1] * self.annualization)
            else:
                m1 = max(self.df.loc[:, actif1].first_valid_index(), self.df.loc[:, actif2].first_valid_index())
                while self.df.loc[m1:, actif1].isna().any() or self.df.loc[m1:, actif2].isna().any():
                    m1+=1
                constante1 = self.df.loc[0:len(self.df.index)-96, [actif1,actif2]].cov().iloc[0,1] * self.annualization
                constante2 = self.df.loc[len(self.df.index)-96:len(self.df.index), [actif1,actif2]].cov().iloc[0,1] * self.annualization
                high = max(constante1, constante2)
                low = min(constante1, constante2)

                if horizon == 1:
                    if (high - low) / low >= 0.3:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(len(self.df.index)-108, m1), len(self.df.index), 6)])
                    else:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(m1, 5), len(self.df.index), 6)])
                        ind_to_remove = np.argsort(X)[-4:]
                        mask = np.ones(len(X), dtype=bool)
                        mask[ind_to_remove] = False
                        X = X[mask]
                elif horizon == 3:
                    if (high - low) / low >= 0.3:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(len(self.df.index)-144, m1), len(self.df.index), 6)])
                    else:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(m1, 5), len(self.df.index), 6)])
                        ind_to_remove = np.argsort(X)[-3:]
                        mask = np.ones(len(X), dtype=bool)
                        mask[ind_to_remove] = False
                        X = X[mask]
                elif horizon == 5 or horizon == 7:
                    if (high - low) / low >= 0.3:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(len(self.df.index)-180, m1), len(self.df.index), 6)])
                    else:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(m1, 5), len(self.df.index), 6)])
                        ind_to_remove = np.argsort(X)[-2:]
                        mask = np.ones(len(X), dtype=bool)
                        mask[ind_to_remove] = False
                        X = X[mask]
                elif horizon == 10:
                    if (high - low) / low >= 0.3:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(len(self.df.index)-240, m1), len(self.df.index), 6)])
                    else:
                        X = np.array([self.df.loc[t-5:t, [actif1,actif2]].cov().iloc[0,1] * self.annualization for t in range(max(m1, 5), len(self.df.index), 6)])
                        ind_to_remove = np.argsort(X)[-1:]
                        mask = np.ones(len(X), dtype=bool)
                        mask[ind_to_remove] = False
                        X = X[mask]

                X = X.reshape(len(X), 1)
                scaler = MinMaxScaler(feature_range=(-1, 1))
                scaler = scaler.fit(X)
                scaled_X = scaler.transform(X)
                train = self.timeseries_to_supervised(scaled_X, lag=1)
                values = train.values
                X, y = values[:, 0].reshape(-1, 1), values[:, 1].reshape(-1, 1)
                X = X.reshape((X.shape[0], 1, X.shape[1]))

                model = self.fit_lstm(X, y, batch_size, nb_epoch, neurons)
                cov_forecast = model.predict(X, batch_size=1)
                cov.append(scaler.inverse_transform(cov_forecast)[:2*horizon, 0].mean())
                k+=1
                if progress_bar:  # Update the progress bar if provided
                    progress_bar.update(1)
        covs=np.zeros((len(self.df.columns),len(self.df.columns)))
        k=0
        for j in range(len(self.df.columns)):
            for i in range(j, len(self.df.columns)):
                covs[j,i]=covs[i,j]=cov[k]
                k+=1
                
        covariance=pd.DataFrame(covs, columns=self.df.columns, index=self.df.columns)
        volatilities = np.sqrt(np.diag(covariance))

        # Parcourir chaque élément de la matrice
        for i in range(covariance.shape[0]):
            for j in range(covariance.shape[1]):
                # Calculer le produit des volatilités correspondantes
                vol_product = volatilities[i] * volatilities[j]
                # Vérifier si la covariance est supérieure au produit des volatilités
                if covariance.iloc[i, j] > vol_product:
                    # Remplacer la valeur par le produit des volatilités
                    covariance.iloc[i, j] = vol_product
        return covariance

    def volatilite(self, covariance):
        vol=[]
        for i in range(covariance.shape[0]):
            vol.append(np.sqrt(covariance.iloc[i,i]))
        return(np.array(vol))
    
    def correlation(self, covariance):
        correl= np.zeros((covariance.shape[0],covariance.shape[0]))
        vol=self.volatilite(covariance)
        for i in range(covariance.shape[0]):
            for j in range(covariance.shape[1]):
                correl[i,j]=covariance.iloc[i,j]/(vol[i]*vol[j])
        return(pd.DataFrame(correl, columns=self.df.columns, index=self.df.columns))